<a href="https://colab.research.google.com/github/armandossrecife/my_validation/blob/main/avalia_modelo_inspecao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Funções de apoio


In [32]:
import pandas as pd
import collections
import matplotlib.pyplot as plt
import seaborn as sns
import random
import scipy.stats as stats
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

def convert_to_upper(list_of_strings):
  upper_list = []
  for string in list_of_strings:
    upper_list.append(string.upper())
  return upper_list

def conta_yes_no(my_list):
  conta_yes, conta_no = 0,0
  for each in my_list:
    if each == 'YES':
      conta_yes += 1
    else:
      conta_no += 1
  return conta_yes, conta_no

def convert_yes_no_to_int(dfa, dfb, my_column):
  labels = ["YES", "NO"]  # Define your possible labels here

  # Convert labels to numerical values
  label_to_num = {label: i for i, label in enumerate(labels)}
  dfa[my_column] = dfa[my_column].map(label_to_num)
  dfb[my_column] = dfb[my_column].map(label_to_num)
  return dfa, dfb

# Define a function to calculate Cohen's Kappa
def calculate_cohens_kappa(dfa, dfb, my_column):
    # Calculate Cohen's Kappa
    kappa = cohen_kappa_score(dfa[my_column], dfb[my_column])
    return kappa

def extra_metrics_yes(dfa, dfb, my_column):
  # Define your ground truth (actual values) and predicted values
  ground_truth = dfa[my_column]  # Ground truth values from manual inspection
  predicted = dfb[my_column]     # Predicted values from ChatGPT inspection

  # Calculate precision, recall, accuracy, and F1-score
  precision = precision_score(ground_truth, predicted, pos_label=1)
  recall = recall_score(ground_truth, predicted, pos_label=1)
  accuracy = accuracy_score(ground_truth, predicted)
  f1 = f1_score(ground_truth, predicted, pos_label=1)
  return precision, recall, accuracy, f1

# Importa dados originais

Merge da inspeção manual e inspeção via ChatGPt

In [46]:
df_inspection = pd.read_excel('inspecao_merge_manual_chatgpt.xlsx')
df_inspection.sample(5)

,issue_key,issue_type,summary,description,architecture_issue,i2_chatgpt_ai,TP,TN,FP,FN
97,CASSANDRA-17147,New Feature,Guardrails prototype,The purpose of this ticket is creating an init...,Yes,Yes,1,0,0,0
101,CASSANDRA-17034,Improvement,CEP-11: Memtable API implementation,Pluggable memtable API as described in [CEP-11...,Yes,Yes,1,0,0,0
74,CASSANDRA-2478,New Feature,Custom CQL protocol/transport,A custom wire protocol would give us the flexi...,Yes,Yes,1,0,0,0
77,CASSANDRA-2069,Bug,Read repair causes tremendous GC pressure,"To reproduce: start a three node cluster, inse...",Yes,No,0,0,0,1
47,CASSANDRA-6230,Sub-task,Write hints to flat files instead of the syste...,Writing to a file would have less overhead on ...,Yes,Yes,1,0,0,0


In [47]:
df_inspection_check = df_inspection[['issue_key', 'issue_type', 'summary', 'architecture_issue', 'i2_chatgpt_ai', 'TP', 'TN', 'FP', 'FN']]
df_inspection_check.sample(10)

,issue_key,issue_type,summary,architecture_issue,i2_chatgpt_ai,TP,TN,FP,FN
121,CASSANDRA-15066,Improvement,Improvements to Internode Messaging,Yes,Yes,1,0,0,0
141,CASSANDRA-13457,Sub-task,Diag. Events: Add base classes,Yes,Yes,1,0,0,0
140,CASSANDRA-1391,Improvement,Allow Concurrent Schema Migrations,Yes,Yes,1,0,0,0
11,CASSANDRA-9402,Task,Implement proper sandboxing for UDFs,Yes,Yes,1,0,0,0
69,CASSANDRA-3578,Improvement,Multithreaded commitlog,Yes,Yes,1,0,0,0
64,CASSANDRA-3761,New Feature,CQL 3.0,Yes,Yes,1,0,0,0
47,CASSANDRA-6230,Sub-task,Write hints to flat files instead of the syste...,Yes,Yes,1,0,0,0
77,CASSANDRA-2069,Bug,Read repair causes tremendous GC pressure,Yes,No,0,0,0,1
34,CASSANDRA-7396,New Feature,Allow selecting Map values and Set elements,No,No,0,1,0,0
54,CASSANDRA-5403,Bug,"Clean up ColumnFamily, ISortedColumns heirarchy",Yes,Yes,1,0,0,0


# Dados da inspeção manual

In [49]:
df_inspection_check_manual = df_inspection_check[['issue_key', 'issue_type','summary','architecture_issue']]
df_inspection_check_manual['architecture_issue'] = df_inspection_check_manual['architecture_issue'].str.upper()
df_inspection_check_manual

<ipython-input-49-516850d7e04a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inspection_check_manual['architecture_issue'] = df_inspection_check_manual['architecture_issue'].str.upper()


,issue_key,issue_type,summary,architecture_issue
0,CASSANDRA-9975,Sub-task,Flatten Iterator call hierarchy with a shared ...,YES
1,CASSANDRA-994,Task,re-organize endpointsnitch implementations,YES
2,CASSANDRA-9932,Improvement,Make all partitions btree backed,YES
3,CASSANDRA-9921,Improvement,Combine MV schema definition with MV table def...,YES
4,CASSANDRA-9705,Sub-task,Simplify some of 8099's concrete implementations,YES
...,...,...,...,...
157,CASSANDRA-1072,Sub-task,Increment counters,YES
158,CASSANDRA-10661,Improvement,Integrate SASI to Cassandra,YES
159,CASSANDRA-1034,Bug,Remove assumption that Key to Token is one-to-one,YES
160,CASSANDRA-10109,Sub-task,Windows dtest 3.0: ttl_test.py failures,YES


In [50]:
df_inspection_check_manual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   issue_key           162 non-null    object
 1   issue_type          162 non-null    object
 2   summary             162 non-null    object
 3   architecture_issue  162 non-null    object
dtypes: object(4)
memory usage: 5.2+ KB


In [51]:
ai_manual = df_inspection_check_manual.architecture_issue.to_list()
conta_yes, conta_no = conta_yes_no(my_list=ai_manual)
print(f'Soma Yes and No: {conta_yes+conta_no}, Yes: {conta_yes}, No: {conta_no}')
print(f'%Yes: {round((conta_yes/(conta_yes+conta_no)*100),2)}, %No: {round((conta_no/(conta_yes+conta_no)*100),2)}')

Soma Yes and No: 162, Yes: 136, No: 26
%Yes: 83.95, %No: 16.05


# Dados da inspeção via ChatGPT

In [52]:
df_inspection_check_chagpt = df_inspection_check[['issue_key', 'issue_type','summary','i2_chatgpt_ai']]
df_inspection_check_chagpt['architecture_issue'] = df_inspection_check_chagpt['i2_chatgpt_ai']
df_inspection_check_chagpt['architecture_issue'] = df_inspection_check_chagpt['architecture_issue'].str.upper()
df_inspection_check_chagpt = df_inspection_check_chagpt[['issue_key', 'issue_type', 'summary', 'architecture_issue']]
df_inspection_check_chagpt.sample(10)

<ipython-input-52-d2a9498a21d6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inspection_check_chagpt['architecture_issue'] = df_inspection_check_chagpt['i2_chatgpt_ai']
<ipython-input-52-d2a9498a21d6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inspection_check_chagpt['architecture_issue'] = df_inspection_check_chagpt['architecture_issue'].str.upper()


,issue_key,issue_type,summary,architecture_issue
10,CASSANDRA-9425,Sub-task,Make node-local schema fully immutable,YES
136,CASSANDRA-14115,Sub-task,Refactor streaming,YES
153,CASSANDRA-11580,Sub-task,remove DatabaseDescriptor dependency from Segm...,YES
87,CASSANDRA-18134,Improvement,Improve handling of min/max clustering in sstable,NO
45,CASSANDRA-6379,New Feature,Replace index_interval with min/max_index_inte...,YES
2,CASSANDRA-9932,Improvement,Make all partitions btree backed,YES
137,CASSANDRA-14092,Bug,Max ttl of 20 years will overflow localDeletio...,YES
34,CASSANDRA-7396,New Feature,Allow selecting Map values and Set elements,NO
142,CASSANDRA-13426,Sub-task,Make all DDL statements idempotent and not dep...,YES
36,CASSANDRA-7066,Improvement,Simplify (and unify) cleanup of compaction lef...,NO


In [53]:
df_inspection_check_chagpt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   issue_key           162 non-null    object
 1   issue_type          162 non-null    object
 2   summary             162 non-null    object
 3   architecture_issue  162 non-null    object
dtypes: object(4)
memory usage: 5.2+ KB


In [54]:
ai_chatgpt = df_inspection_check_chagpt.architecture_issue.to_list()
conta_yes, conta_no = conta_yes_no(my_list=ai_chatgpt)
print(f'Soma Yes and No: {conta_yes+conta_no}, Yes: {conta_yes}, No: {conta_no}')
print(f'%Yes: {round((conta_yes/(conta_yes+conta_no)*100),2)}, %No: {round((conta_no/(conta_yes+conta_no)*100),2)}')

Soma Yes and No: 162, Yes: 123, No: 39
%Yes: 75.93, %No: 24.07


# Cálculo do Kappa


In [55]:
# Converte a coluna architecture_issue para 0 (NO) ou 1 (YES)
dfa, dfb = convert_yes_no_to_int(dfa=df_inspection_check_manual, dfb=df_inspection_check_chagpt, my_column='architecture_issue')

<ipython-input-33-2224af11d2a5>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa[my_column] = dfa[my_column].map(label_to_num)


In [56]:
kappa_manual_chatgpt = calculate_cohens_kappa(dfa=df_inspection_check_manual, dfb=df_inspection_check_chagpt, my_column='architecture_issue')
print(round(kappa_manual_chatgpt, 3))

0.752


# Cálculo da performance do modelo de inspeção ChatGPT

In [58]:
my_precision, my_recall, my_accuracy, my_f1_score = extra_metrics_yes(dfa=df_inspection_check_manual, dfb=df_inspection_check_chagpt, my_column='architecture_issue')
print(f'my_precision: {round(my_precision, 3)}, my_recall: {round(my_recall, 3)}, my_accuracy: {round(my_accuracy, 3)}, my_f1_score: {round(my_f1_score,3)}')

my_precision: 0.667, my_recall: 1.0, my_accuracy: 0.92, my_f1_score: 0.8
